## Tarea 2 (10 puntos). Operaciones con RDDs
- Obtener información de algún origen de datos (propio o de API)
- Convertir el origen de datos a RDD con pySpark
- Realizar alguna operación en el RDD, como estadísticas descriptivas básicas
- Subir la práctica a un repositorio público y etiquetarla claramente

In [2]:
import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from pyspark.sql import SparkSession

In [3]:
import multiprocessing

multiprocessing.cpu_count()

16

In [4]:
from pyspark import SparkConf

In [6]:
conf = SparkConf().setAppName("Tarea2")\
    .set("spark.master","local[10]")\
    .set("spark.executor.cores","2")\
    .set("spark.executor.memory", "4g")

In [7]:
#sc = SparkContext("local", "RDDs")
sc = SparkContext(conf= conf)

In [8]:
spark = SparkSession(sparkContext= sc)


In [10]:
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [11]:
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\data.zip')

path_guardar = pathlib.Path.joinpath(direccion.parent.parent, 'Datos')

archivos_datos = [
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2022.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2021.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2020.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2019.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2018.csv').as_posix()
]

archivos_datos_en_zip  = [
    'Combined_Flights_2022.csv',
    'Combined_Flights_2021.csv',
    'Combined_Flights_2020.csv',
    'Combined_Flights_2019.csv',
    'Combined_Flights_2018.csv'
]


In [18]:
with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
    for archivo in archivos_datos_en_zip:
        archivo_zip.extract(archivo, path_guardar)

In [13]:
def leer_archivo(archivo):
    direccion = pathlib.Path.cwd()
    direccion_archivo =  pathlib.Path.joinpath(direccion.parent.parent, 'Datos',archivo)
    
    return spark.read.csv(direccion_archivo)

In [24]:
rdd = spark.sparkContext.parallelize(archivos_datos)
rdd_contenido = rdd.map(lambda archivo: {"archivo": archivo,"datos": leer_archivo(archivo)})


In [20]:
rdd = spark.sparkContext.parallelize(archivos_datos)
rdd_contenido = rdd.map(lambda archivo: leer_archivo(archivo))

In [12]:
from pyspark import StorageLevel

In [17]:
df = spark.read.option('header','true').csv(archivos_datos).select(columnas)

rdd = df.rdd.coalesce(5)
rdd = rdd.persist(StorageLevel.MEMORY_AND_DISK)

In [18]:
rdd.getNumPartitions()

5

In [19]:
rdd

CoalescedRDD[33] at coalesce at DirectMethodHandleAccessor.java:103

In [20]:
rdd.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (Andree executor driver): java.net.SocketException: Connection reset
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:318)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:346)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:796)
	at java.base/java.net.Socket$SocketInputStream.implRead(Socket.java:1108)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1095)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:291)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:347)
	at java.base/java.io.BufferedInputStream.implRead(BufferedInputStream.java:420)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:39